# SpaceRocks

### Vectorized coordinate transformation and ephemeris calculation with robust unit handling.

To install, simply `pip install spacerocks`

In [ ]:
from spacerocks import SpaceRock
import numpy as np

All of the methods have docstrings in here.

In [ ]:
help(SpaceRock)

Define the distribution of objects. For Keplerian coordinates, you can use either epoch or mean anomaly.

In [ ]:
N = int(1e3)
a = np.random.uniform(30, 31, N)
e = np.random.uniform(0, 0.5, N)
inc = np.random.rayleigh(np.pi/15, N)
arg = np.random.uniform(0, 2*np.pi, N)
node = np.random.uniform(0, 2*np.pi, N)
obs_date = np.ones(N) * 2480000
H = np.random.uniform(4, 7, N)

# It doesn't matter which one you use.
epoch = np.random.uniform(2400000, 2480000, N)
M = np.random.uniform(0, 2*np.pi, N)

Call the class SpaceRock. The default settings are as described in the help page. All arguments and keyword arguments are case-insensitive.

- input_coordinates can be Keplerian or Cartesian
- input_frame can be Barycentric or Heliocentric 
- input_angles can be radians or degrees
- NSIDE is a list (or an array) of the NSIDE healpix values you want calculated.

It's vectorized, so you can pass in as many objects as you want. Note that tau is the observation date.

In [ ]:
rocks = SpaceRock(a=a, 
                  e=e, 
                  inc=inc, 
                  arg=arg, 
                  node=node, 
                  epoch=epoch, 
                  tau=obs_date,
                  H=H,
                  name=['rock{}'.format(idx) for idx in range(N)], # default is a random 10-digit integer.
                  NSIDE=[128, 256, 1024],
                  precise=False,
                  input_coordinates='keplerian',
                  input_frame='barycentric',
                  input_angles='radians')

You can get a look at the data by writing it to either a pandas dataframe of an astropy table. In my opinion pandas dataframes are nicer to work with, but they don't support astropy units (yet).

In [ ]:
rocks_df = rocks.pandas_df()
rocks_df.head()

In [ ]:
rocks_table = rocks.astropy_table()
rocks_table[:5]

You can also pass in just a single object. I'll try BP519. Here I've turned on precision. This involves a non-vectorized solution to Kepler's equation to calculate the eccentric anomaly, and a topocentric correction to the Earth's position. Right now, only DECam is supported, but I'm working on implementing an obscode feature.

In [ ]:
BP = SpaceRock(a=4.490784528803734E+02, 
               e=9.215119038599956E-01, 
               inc=5.411067896592579E+01, 
               arg=3.480604843629308E+02, 
               node=1.352131890380128E+02, 
               epoch=2458285.5, 
               tau=2458385.5,
               H=4.4, 
               name='BP',
               precise=True,
               input_coordinates='keplerian',
               input_frame='barycentric',
               input_angles='degrees')

Printing out the results:

In [ ]:
BP.pandas_df()

You can also go between heliocentric and barycentric coordinates with the to_helio() and to_bary() methods.

In [ ]:
BP_helio = BP.to_helio()
BP_helio.pandas_df()

In [ ]:
BP_bary = BP_helio.to_bary()
BP_bary == BP

You can predict the position of a lot of objects at a given date

In [ ]:
rocks = SpaceRock(a=a, 
                  e=e, 
                  inc=inc, 
                  arg=arg, 
                  node=node, 
                  epoch=epoch, 
                  tau=obs_date,
                  H=H,
                  name=['rock{}'.format(idx) for idx in range(N)], # default is a random 10-digit integer.
                  NSIDE=[128, 256, 1024],
                  precise=False,
                  input_coordinates='keplerian',
                  input_frame='barycentric',
                  input_angles='radians')

rocks_tomorrow = rocks.predict(2458385.5)
rocks_tomorrow.pandas_df().head()

Or if you want, you can predice the position of a lot of objects at a lot of different dates.

In [ ]:
rocks_on_many_days = rocks.predict(np.random.uniform(2458385.5, 2868985.5, N))
rocks_on_many_days.pandas_df().head()

You can calculate the predicted position of the same object at many dates. This could probably be cleaner, but it may take some doing to implement.

In [ ]:
N = 100
BP = SpaceRock(a=np.repeat(4.490784528803734E+02, N), 
               e=np.repeat(9.215119038599956E-01, N), 
               inc=np.repeat(5.411067896592579E+01, N), 
               arg=np.repeat(3.480604843629308E+02, N), 
               node=np.repeat(1.352131890380128E+02, N), 
               epoch=np.repeat(2458285.5, N), 
               tau=np.linspace(2058285.5, 2458285.5, N),
               H=np.repeat(4.4, N), 
               name=['rock{}'.format(idx) for idx in range(N)],
               precise=False,
               input_coordinates='keplerian',
               input_frame='barycentric',
               input_angles='degrees')

You quickly plot the equatorial position. 

In [ ]:
BP.plot_radec('red')

If you want to keep your calculations, you can write them to a file like this.

In [ ]:
path = '/wherever/you/want/the/file.csv'
BP.write_to_csv(path)